In [20]:
!pip install openpyxl

import openpyxl
import pandas as pd
import os
import re

On va procéder ici au merging des bases de données. 

## Préparation des sites 

### Préparation de la base manuelle

In [21]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(base_dir, 'data', 'text', 'manuel.xlsx')

# Chargez le fichier Excel dans un DataFrame
sites = pd.read_excel(fichier_excel, sheet_name=0)

In [22]:
#On garde les colonnes intéressantes 
sites = sites[['Id', 'nom', 'site', 'Idéologie']]
sites = sites.dropna(subset=['nom'])
sites

,Id,nom,site,Idéologie
0,1,Renaissance Catholique,https://renaissancecatholique.fr,Intégrisme Catholique
1,2,AGRIF,https://www.lagrif.fr/,"Islamophobie, Nationalisme"
2,3,Génération Zemmour,https://www.generation-zemmour.fr,"Nationaliste, Identitaire"
3,4,ISSEP,https://www.issep.fr/,Extrême-Droite
4,5,Egalité et Réconciliation,https://www.egaliteetreconciliation.fr,"Antisémitisme, Identitaire"
5,6,La Porte Latine,https://laportelatine.org/,Intégrisme Catholique
6,7,Rassemblement National,https://rassemblementnational.fr/,Extrême-Droite
7,8,Action Française,https://www.lactionfrancaise.fr,Monarchiste
8,9,Riposte Catholique,https://riposte-catholique.fr/,Intégrisme Catholique
9,10,Les Ruminants,https://lesruminants.com/,"Anarchisme, Ecologie, TERF"


In [23]:
articles = pd.read_excel(fichier_excel, sheet_name=1) #Récupération articles
articles = articles.dropna(subset=['contenu']) #Fuck les NaN
nouveaux_noms = {
    'URL ': 'URL',
}
articles = articles.rename(columns=nouveaux_noms)

### Adaptation de Femellistes 

In [24]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'femelliste_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
femiste = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

In [25]:
femiste = femiste.drop('Catégories', axis=1, errors='ignore')
femiste['ID ASSO'] = 12 #On ajoute l'ID ASSO poue le merge futur 

# Renommer les colonnes pour faciliter la concaténation

nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}

femiste = femiste.rename(columns=nouveaux_noms)
femiste

,Titre,Auteur,Date,Extrait,contenu,URL,ID ASSO
0,Le transgenrisme instrumentalise la cause inte...,Alice,06/10/2023,Notre contributrice Alice (@libre.et.impolie s...,Le transgenrisme instrumentalise la cause inte...,https://www.femelliste.com/articles-femellisme...,12
1,Comment le transgenrisme et la culture queer i...,FEMELLISTE,26/06/2023,"Sur les réseaux sociaux, nous avons remarqué q...",Comment le transgenrisme et la culture queer i...,https://www.femelliste.com/articles-femellisme...,12
2,Entretien avec le collectif “Féminicides par c...,Marguerite Stern,21/04/2023,"Elles sont quatre. Depuis 2016, elles comptabi...",Entretien avec le collectif “Féminicides par c...,https://www.femelliste.com/articles-femellisme...,12
3,Tania : mère d’une adolescente anciennement tr...,Marguerite Stern,30/03/2023,Tania (prénom modifié) est éducatrice sociale ...,Tania : mère d’une adolescente anciennement tr...,https://www.femelliste.com/articles-femellisme...,12
4,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",Marguerite Stern,14/03/2023,On l’adore. Et comme elle vient de publier son...,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",https://www.femelliste.com/articles-femellisme...,12
5,"Bloqueurs de puberté : au journal le Monde, pe...",Zest De Gravity,18/02/2023,"Le 5 décembre 2022, le journal Le Monde publia...","Bloqueurs de puberté : au journal le Monde, pe...",https://www.femelliste.com/articles-femellisme...,12
6,Canada : Un lycéen exclu et arrêté par la poli...,Marguerite Stern,13/02/2023,Le lycée catholique St. Joseph’s en Ontario (C...,Canada : Un lycéen exclu et arrêté par la poli...,https://www.femelliste.com/articles-femellisme...,12
7,Quand la CAF fait la promotion du transgenrisme.,Marguerite Stern,02/02/2023,"Dans un article publié le 25 Janvier 2023, la ...",Quand la CAF fait la promotion du transgenrism...,https://www.femelliste.com/articles-femellisme...,12
8,États-unis : un homme jugé coupable de féminic...,Marguerite Stern,31/01/2023,L’information provient du média anglophone Red...,États-unis : un homme jugé coupable de féminic...,https://www.femelliste.com/articles-femellisme...,12
9,Royaume-uni : la loi concernant l’incarcératio...,Marguerite Stern,26/01/2023,Selon un communiqué publié par le site officie...,Royaume-uni : la loi concernant l’incarcératio...,https://www.femelliste.com/articles-femellisme...,12


### Adaptation de Tradfem

In [26]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'tradfem_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
tradfem = pd.read_csv(fichier ,sep='\t', encoding='utf-8')
tradfem

,Titre,Date,Auteur,Image URL,Texte Complet,URL Complète
0,« Accorder priorité à l’identité de genre sur ...,Il y a 13 heures 03/12/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/12/ph...,« Accorder priorité à l’identité de genre sur ...,https://tradfem.wordpress.com/2023/12/03/accor...
1,Les féminicides se multiplient au Canada… et a...,Il y a 4 jours 30/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/st...,Les féminicides se multiplient au Canada… et a...,https://tradfem.wordpress.com/2023/11/30/les-f...
2,Oscar Pistorius est libéré après avoir purgé m...,Il y a 1 semaine 24/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/ph...,Oscar Pistorius est libéré après avoir purgé m...,https://tradfem.wordpress.com/2023/11/24/oscar...
3,« Pourquoi fait-il ça? » est maintenant offert...,17/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/co...,« Pourquoi fait-il ça? » est maintenant offert...,https://tradfem.wordpress.com/2023/11/17/pourq...
4,"L’horreur, l’horreur",12/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/is...,"L’horreur, l’horreurTRADFEM/12/11/2023________...",https://tradfem.wordpress.com/2023/11/12/lhorr...
...,...,...,...,...,...,...
112,Le prix Pulitzer décerné à l’écrivain Andrea L...,11/05/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/05/ph...,Le prix Pulitzer décerné à l’écrivain Andrea L...,https://tradfem.wordpress.com/2023/05/11/le-pr...
113,Lu sur le substack du proféministe Graham Linehan,11/05/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/05/il...,Lu sur le substack du proféministe Graham Line...,https://tradfem.wordpress.com/2023/05/11/lu-su...
114,"La Dre Judith Herman, psychiatre et pionnière ...",11/05/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/05/vo...,"La Dre Judith Herman, psychiatre et pionnière ...",https://tradfem.wordpress.com/2023/05/11/la-dr...
115,Errements de l’adoption internationale,10/05/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/05/co...,Errements de l’adoption internationaleTRADFEM/...,https://tradfem.wordpress.com/2023/05/10/errem...


In [27]:
#Ajout et suppressions de colonnes intéressantes 
tradfem = tradfem.drop(['Image URL'], axis=1, errors='ignore')
tradfem['ID ASSO'] = 13 #On ajoute l'ID ASSO poue le merge futur 

#Rename utiles 
nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}

tradfem = tradfem.rename(columns=nouveaux_noms)
tradfem

,Titre,Date,Auteur,contenu,URL,ID ASSO
0,« Accorder priorité à l’identité de genre sur ...,Il y a 13 heures 03/12/2023,TRADFEM,« Accorder priorité à l’identité de genre sur ...,https://tradfem.wordpress.com/2023/12/03/accor...,13
1,Les féminicides se multiplient au Canada… et a...,Il y a 4 jours 30/11/2023,TRADFEM,Les féminicides se multiplient au Canada… et a...,https://tradfem.wordpress.com/2023/11/30/les-f...,13
2,Oscar Pistorius est libéré après avoir purgé m...,Il y a 1 semaine 24/11/2023,TRADFEM,Oscar Pistorius est libéré après avoir purgé m...,https://tradfem.wordpress.com/2023/11/24/oscar...,13
3,« Pourquoi fait-il ça? » est maintenant offert...,17/11/2023,TRADFEM,« Pourquoi fait-il ça? » est maintenant offert...,https://tradfem.wordpress.com/2023/11/17/pourq...,13
4,"L’horreur, l’horreur",12/11/2023,TRADFEM,"L’horreur, l’horreurTRADFEM/12/11/2023________...",https://tradfem.wordpress.com/2023/11/12/lhorr...,13
...,...,...,...,...,...,...
112,Le prix Pulitzer décerné à l’écrivain Andrea L...,11/05/2023,TRADFEM,Le prix Pulitzer décerné à l’écrivain Andrea L...,https://tradfem.wordpress.com/2023/05/11/le-pr...,13
113,Lu sur le substack du proféministe Graham Linehan,11/05/2023,TRADFEM,Lu sur le substack du proféministe Graham Line...,https://tradfem.wordpress.com/2023/05/11/lu-su...,13
114,"La Dre Judith Herman, psychiatre et pionnière ...",11/05/2023,TRADFEM,"La Dre Judith Herman, psychiatre et pionnière ...",https://tradfem.wordpress.com/2023/05/11/la-dr...,13
115,Errements de l’adoption internationale,10/05/2023,TRADFEM,Errements de l’adoption internationaleTRADFEM/...,https://tradfem.wordpress.com/2023/05/10/errem...,13


## Merging

In [28]:
df = pd.concat([articles, femiste, tradfem], ignore_index=True, sort=False)
df

,ID ASSO,URL,Type de document,ID ARTICLE,contenu,Titre,Auteur,Date,Extrait
0,1.0,https://renaissancecatholique.fr/blog/les-just...,Article d'opinion,1.0,"Par jugement du 23 novembre 2021, la XVIIème c...",NaN,NaN,NaN,NaN
1,3.0,https://www.generation-zemmour.fr/le-funeste-i...,Article d'opinion,2.0,Le funeste inventaire de la loi bioéthique ado...,NaN,NaN,NaN,NaN
2,3.0,https://www.generation-zemmour.fr/1568,Article d'opinion,3.0,"Rousseau, Woerth et les wokes : Quand les Lumi...",NaN,NaN,NaN,NaN
3,3.0,https://www.generation-zemmour.fr/le-feminisme...,Article d'opinion,4.0,Le féminisme progressiste actuel défend-il tou...,NaN,NaN,NaN,NaN
4,3.0,https://www.generation-zemmour.fr/le-pronom-ie...,Article d'opinion,5.0,Le pronom « iel » ou le sexe des anges\nLa thé...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
174,13.0,https://tradfem.wordpress.com/2023/05/11/le-pr...,NaN,NaN,Le prix Pulitzer décerné à l’écrivain Andrea L...,Le prix Pulitzer décerné à l’écrivain Andrea L...,TRADFEM,11/05/2023,NaN
175,13.0,https://tradfem.wordpress.com/2023/05/11/lu-su...,NaN,NaN,Lu sur le substack du proféministe Graham Line...,Lu sur le substack du proféministe Graham Linehan,TRADFEM,11/05/2023,NaN
176,13.0,https://tradfem.wordpress.com/2023/05/11/la-dr...,NaN,NaN,"La Dre Judith Herman, psychiatre et pionnière ...","La Dre Judith Herman, psychiatre et pionnière ...",TRADFEM,11/05/2023,NaN
177,13.0,https://tradfem.wordpress.com/2023/05/10/errem...,NaN,NaN,Errements de l’adoption internationaleTRADFEM/...,Errements de l’adoption internationale,TRADFEM,10/05/2023,NaN


In [29]:
#Refaire des nombres de référence des articles
df['ID ARTICLE'] = range(1, len(df) + 1)
df

,ID ASSO,URL,Type de document,ID ARTICLE,contenu,Titre,Auteur,Date,Extrait
0,1.0,https://renaissancecatholique.fr/blog/les-just...,Article d'opinion,1,"Par jugement du 23 novembre 2021, la XVIIème c...",NaN,NaN,NaN,NaN
1,3.0,https://www.generation-zemmour.fr/le-funeste-i...,Article d'opinion,2,Le funeste inventaire de la loi bioéthique ado...,NaN,NaN,NaN,NaN
2,3.0,https://www.generation-zemmour.fr/1568,Article d'opinion,3,"Rousseau, Woerth et les wokes : Quand les Lumi...",NaN,NaN,NaN,NaN
3,3.0,https://www.generation-zemmour.fr/le-feminisme...,Article d'opinion,4,Le féminisme progressiste actuel défend-il tou...,NaN,NaN,NaN,NaN
4,3.0,https://www.generation-zemmour.fr/le-pronom-ie...,Article d'opinion,5,Le pronom « iel » ou le sexe des anges\nLa thé...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
174,13.0,https://tradfem.wordpress.com/2023/05/11/le-pr...,NaN,175,Le prix Pulitzer décerné à l’écrivain Andrea L...,Le prix Pulitzer décerné à l’écrivain Andrea L...,TRADFEM,11/05/2023,NaN
175,13.0,https://tradfem.wordpress.com/2023/05/11/lu-su...,NaN,176,Lu sur le substack du proféministe Graham Line...,Lu sur le substack du proféministe Graham Linehan,TRADFEM,11/05/2023,NaN
176,13.0,https://tradfem.wordpress.com/2023/05/11/la-dr...,NaN,177,"La Dre Judith Herman, psychiatre et pionnière ...","La Dre Judith Herman, psychiatre et pionnière ...",TRADFEM,11/05/2023,NaN
177,13.0,https://tradfem.wordpress.com/2023/05/10/errem...,NaN,178,Errements de l’adoption internationaleTRADFEM/...,Errements de l’adoption internationale,TRADFEM,10/05/2023,NaN


In [31]:
#On exporte 
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_def.csv'), index=False)